## Tkinter GUI

In [1]:
from tkinter import *
from tkinter import messagebox, filedialog, messagebox, ttk
from PIL import ImageTk, Image
import csv

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib qt

In [167]:
#new
# Data and Starting App
data = {
    'Months': ['January','February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'],
    'Year_Sales_2021': ['-','-','-','-','-','-','-','-','-','-','-','-'],
    #for MAD error calculations
    '2022_Actual_Sales': ['-','-','-','-','-','-','-','-','-','-','-','-']
}
df = pd.DataFrame(data= data)

root = Tk()
root.title("Sales Forecasting Device")

Title_Label = Label(root, bg="gray", padx=90, text="Sales Forecasting Device")
Title_Label.grid(row=0, column=0, columnspan = 5)

Window_Size_After_Inputs = "975x560"

#Global Frames (put here so that it can be errased off the tab if needed)
Input_Sales_Frame = LabelFrame(root, pady= 6, padx=15)
Input_Error = Label(Input_Sales_Frame, text="Input all fields as correct number \n(not a string or a negative number)", fg="red", bg="lightgray")
Empty_Label = Label(Input_Sales_Frame, text="")
Naive_Frame = LabelFrame(root, pady= 6, padx=15)
MA_Frame = LabelFrame(root, pady= 6, padx=15)
WMA_Frame = LabelFrame(root, pady= 6, padx=15)
EMA_Frame = LabelFrame(root, pady= 6, padx=15)


Display_Data_Frame = LabelFrame(MA_Frame, text="Data Display")
Graph_Comparison_Button = Button(MA_Frame, text="Graph Real 2022 vs Forecasted 2022 Data", bg="lightblue", command= lambda: MACompareGraph()) 
MA_MAD_Label = Label(MA_Frame, text= "edit this down below", font= ('Helvetica 13 underline'))
tv2 = ttk.Treeview(Display_Data_Frame, height = 12)


############################################################################
NavBar_Frame = LabelFrame(root)
#Displays select forecast text only when no forecast option is selected
Select_Option_Label = Label(NavBar_Frame, text="Please Select One of the Above Forecasting Options", pady=30, font= ('Helvetica 13 bold'))
def NavBarSection():
    NavBar_Frame.grid(row=1, column=0, columnspan = 5)
    #Makes Navbar Buttons
    Input_Sales_Button = Button(NavBar_Frame, text= "Reset Inputs", width=23, command= lambda: [InputSalesSection()])
    Naive_Forecast = Button(NavBar_Frame, text= "Naive", width=23, command= NaiveSection)
    MA_Forecast = Button(NavBar_Frame, text= "Moving Average", width=23, command= MASection)
    WMA_Forecast = Button(NavBar_Frame, text= "Weighted Moving Average", width=23, command= WMASection)
    Exponential_Forecast = Button(NavBar_Frame, text= "Exponential", width=23, command= ExponentialSection)
    #Display it
    Input_Sales_Button.grid(row=1, column=0)
    Naive_Forecast.grid(row=1, column=1)
    MA_Forecast.grid(row=1, column=2)
    WMA_Forecast.grid(row=1, column=3)
    Exponential_Forecast.grid(row=1, column=4)
    
    #displays select forecast text
    Select_Option_Label.grid(row= 20, column= 0, columnspan = 5)

############################################################################
def InputSalesSection():
    #hides anything that isnt for this tab from the screen
    Input_Error.grid_forget()
    Empty_Label.grid_forget()
    Select_Option_Label.grid_forget()
    Naive_Frame.grid_forget()
    MA_Frame.grid_forget()
    WMA_Frame.grid_forget()
    EMA_Frame.grid_forget()
    
    # changes size of window after inputing data first time
    window_size_after = "975x160"
    if root.geometry().split('+')[0] == window_size_after:
        root.geometry(Window_Size_After_Inputs)
    
    #Input Sales Section
    Input_Sales_Title_Label = Label(Input_Sales_Frame, text="Input every Month's Sales", font= ('Helvetica 10 underline'))
    Input_Sales_Frame.grid(row=2,column=0, columnspan = 5)
    Input_Sales_Title_Label.grid(row=0, column=0, columnspan = 5)
    Input_Sales_2021_Label = Label(Input_Sales_Frame, text="2021 (for forecasting)")
    Input_Sales_2022_Label = Label(Input_Sales_Frame, text="2022 (for MAD)")
    Input_Sales_2021_Label.grid(row=1, column=1, columnspan = 1)
    Input_Sales_2022_Label.grid(row=1, column=2, columnspan = 2)
    ##########################################
        #Input Texts Section
    rowStart = 1
    columnStart = 0
    d_2021 = {}
    d_2022 = {}

    for i in range(len(df["Months"])):

        rowStart = rowStart + 1
        #2021 Inputs
        Months_Label = Label(Input_Sales_Frame, text= df["Months"][i], padx=20)
        Months_Label.grid(row= rowStart+1, column = columnStart)

        d_2021["Months_2021_Input{0}".format(i)] = Entry(Input_Sales_Frame)
        d_2021["Months_2021_Input{0}".format(i)].grid(row= rowStart+1, column = columnStart+1)

        #2022 Inputs
        d_2022["Months_2022_Input{0}".format(i)] = Entry(Input_Sales_Frame)
        d_2022["Months_2022_Input{0}".format(i)].grid(row= rowStart+1, column = columnStart+2)
    ##########################################
    #Submit Section
    
    #choose CSV File
    CSV_File_Name = Label(Input_Sales_Frame ,text="No File Selected")
    def ChooseCSVFile():
        Input_Sales_Frame.filename = filedialog.askopenfilename(initialdir="/C", title="Select CSV File", filetypes=(("CSV Files", "*.csv"),))
        CSV_File_Name["text"] = Input_Sales_Frame.filename
        global File_Path
        File_Path = CSV_File_Name["text"]
        
        #try to read CSV file
        try:
            CSV_Filename = r"{}".format(File_Path)
            global df
            df = pd.read_csv(File_Path)
            df.columns= ["Months", "Year_Sales_2021", "2022_Actual_Sales"]
            months = ['January','February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
            for i in range(len(months)):
                df["Months"][i] = months[i]
            
            #checks if values are integers and valid or not
            try:
                for i in range(len(df["Year_Sales_2021"])):
                    float(df["Year_Sales_2021"][i])
                    float(df["2022_Actual_Sales"][i])
                
                Submit_Sales = messagebox.showinfo("Submission Successful", "The data has been submitted successfully")
                if Submit_Sales == "ok":
                    root.geometry(window_size_after)
                    NavBarSection()
                    Input_Sales_Frame.grid_forget()
                    
                    #Remove TreeView after changing data
                    Display_Data_Frame.grid_forget()
                    MA_MAD_Label.grid_forget()
                    Graph_Comparison_Button.grid_forget()
                    
            except ValueError:
                Empty_Label.grid(row= 15, column=0, columnspan = 3)
                #displays input error
                Input_Error.config(text="The data inside the CSV is not valid \n (Make sure the data are numbers)")
                Input_Error.grid(row= 16, column=0, columnspan = 3)

                #only changes size of window if its first section of app
                if root.geometry().split('+')[0] == "503x523":
                    root.geometry("503x593")
                else:
                    root.geometry("975x630")
                
        except ValueError:
            #displays Image
            Error_Image = ImageTk.PhotoImage(Image.open("formatExample.jpg"))
            Image_Window = Toplevel()
            Image_Window.wm_transient(root)
            x = root.winfo_x()
            y = root.winfo_y()
            Image_Window.geometry("+%d+%d" % (x + 100, y + 200))
            Error_Image_Label = Label(Image_Window, image= Error_Image)
            Error_Image_Label.pack()
            
            #displays error
            Error_Message = messagebox.showerror("Submission Unsuccessful", "Make sure the CSV File only has 3 Columns with a title and the data similar to this format in the newly opened window")
            if Error_Message == "ok":
                Image_Window.destroy()
                Empty_Label.grid(row= 15, column=0, columnspan = 3)
                #displays input error
                Input_Error.config(text= " The file you have entered is not valid")
                Input_Error.grid(row= 16, column=0, columnspan = 3)
                
                #only changes size of window if its first section of app
                if root.geometry().split('+')[0] == "503x523":
                    root.geometry("503x593")
                    
        except FileNotFoundError:
            Empty_Label.grid(row= 15, column=0, columnspan = 3)
            #displays input error
            Input_Error.config(text= " Could not find File, please try again")
            Input_Error.grid(row= 16, column=0, columnspan = 3)
            
            #only changes size of window if its first section of app
            if root.geometry().split('+')[0] == "503x523":
                root.geometry("503x593")
            else:
                root.geometry("975x630")
        
    def skip():
        Submit_Sales = messagebox.showinfo("Submission Successful", "The data has been submitted successfully")
        if Submit_Sales == "ok":
            root.geometry(window_size_after)
            NavBarSection()
            df["Year_Sales_2021"] = np.random.randint(1,200, size=len(df))
            df["2022_Actual_Sales"] = np.random.randint(1,200, size=len(df))
            Input_Sales_Frame.grid_forget()
            
            #Remove TreeView after changing data
            Display_Data_Frame.grid_forget()
            MA_MAD_Label.grid_forget()
            Graph_Comparison_Button.grid_forget()

    #Validates Inputs of Sales if its integer and Submits if successful
    def SaleSubmit():
        try:
            for i in range(len(df["Year_Sales_2021"])):
                Months_Input_Getter_2021 = (d_2021["Months_2021_Input{0}".format(i)]).get()
                Months_Input_Getter_2022 = (d_2022["Months_2022_Input{0}".format(i)]).get()
                df["Year_Sales_2021"][i] = float(Months_Input_Getter_2021)
                df["2022_Actual_Sales"][i] = float(Months_Input_Getter_2022)

            #Submit successsful window
            Submit_Sales = messagebox.showinfo("Submission Successful", "The data has been submitted successfully")
            if Submit_Sales == "ok":
                root.geometry(window_size_after)
                NavBarSection()
                Input_Sales_Frame.grid_forget()
                
                #Remove TreeView after changing data
                Display_Data_Frame.grid_forget()
                MA_MAD_Label.grid_forget()
                Graph_Comparison_Button.grid_forget()

        except ValueError:
            Empty_Label.grid(row= 15, column=0, columnspan = 3)
            #displays input error
            Input_Error.grid(row= 16, column=0, columnspan = 3)
            
            #only changes size of window if its first section of app
            if root.geometry().split('+')[0] == "503x523":
                root.geometry("503x593")
            else:
                root.geometry("975x630")

#         return Months_Input_Getter_2021, Months_Input_Getter_2022, Submit_Sales

    #Actual Displays
    Choose_CSV_Label = Label(Input_Sales_Frame, text="Or")
    Choose_CSV_Label.grid(row= 18, column=0, columnspan = 3)
    Choose_CSV_Button = Button(Input_Sales_Frame, text="Choose Select CSV File", command= ChooseCSVFile)
    Choose_CSV_Button.grid(row= 19, column=0, columnspan = 3)
    
    #Or Random Assortment of numbers
    skip_Label = Button(Input_Sales_Frame, text="Skip Inputting Data", bg="lightblue", command = skip)
    skip_Label.grid(row= 20, column=0, pady=10)
    
    #Submit Button
    Submit_Button = Button(Input_Sales_Frame, text="Submit", bg="lightgreen", command=SaleSubmit)
    Submit_Button.grid(row= 20, column=2, columnspan=2)
    
#displays it at start of app
InputSalesSection()
############################################################################
#Naive section
def NaiveSection():
    root.geometry(Window_Size_After_Inputs)
    #removes anything that isnt for this tab from the screen
    Input_Error.grid_forget()
    Select_Option_Label.grid_forget()
    Input_Sales_Frame.grid_forget()
    MA_Frame.grid_forget()
    WMA_Frame.grid_forget()
    EMA_Frame.grid_forget()
    
    df["Naive_Forcast"] = df["Year_Sales_2021"]
    #MAD Calculation
    error_df = df.copy()
    error_df["Naive_Error"] = (df["2022_Actual_Sales"] - df["Naive_Forcast"]).abs()
    Naive_MAD = error_df["Naive_Error"].mean()
    
    #Naive Frame
    Naive_Frame.grid(row= 2, column= 0, columnspan= 5)
    
    #frame for TreeView
    Display_Data_Frame = LabelFrame(Naive_Frame, text="Data Display")
    Display_Data_Frame.grid(row= 2, column= 0, columnspan= 5)

    #TreeView Widget
    tv1 = ttk.Treeview(Display_Data_Frame, height = 12)
    tv1["column"] = list(df.columns)
    tv1["show"] = "headings"
    for column in tv1["column"]:
        tv1.heading(column, text = column)

    df_rows = df.to_numpy().tolist()
    for row in df_rows:
        tv1.insert("", "end", values= row)
    tv1.grid(row= 100, column= 0)
    
    Empty_Label = Label(Naive_Frame, text=" ")
    Empty_Label.grid(row=3, column= 0, columnspan= 5)
    #   Graphing and MAD Labels                  lambda used to run function later when running to it (throwaway function)
    Graph_Comparison_Button = Button(Naive_Frame, text="Graph Real 2022 vs Forecasted 2022 Data", bg="lightblue", command= lambda: NaiveCompareGraph()) 
    Naive_MAD_Label = Label(Naive_Frame, text= f"Naive MAD: {Naive_MAD}", font= ('Helvetica 13 underline'))
    Naive_MAD_Label.grid(row=4, column= 0)
    Graph_Comparison_Button.grid(row=4, column= 3)
    def NaiveCompareGraph():
        #Naive Compare
        x= np.array([0,1,2,3,4,5,6,7,8,9,10,11])

        plt.figure(figsize=(10,5), dpi=100)
        plt.plot(error_df["2022_Actual_Sales"], marker=".", markersize="12", markeredgecolor="blue", label="Actual Sales 2022")
        plt.plot(error_df["Naive_Forcast"], marker=".", markersize="12", markeredgecolor="orange", label="Naive Forecasting")
        plt.xticks(x, error_df["Months"], rotation=50)

        plt.title("Actual Sales of 2022 vs Naive Forecasted", fontdict={"fontsize": 15})
        plt.xlabel("Month")
        plt.ylabel("Sales")
       
        plt.legend(bbox_to_anchor=(0.85, 1.1), loc='upper left', borderaxespad=0)


        plt.show()
        
    #when done delete Naive_forecast column
    del df["Naive_Forcast"]
############################################################################
#Moving Average Section
def MASection():
    
    root.geometry("1041x560")
    #removes anything that isnt for this tab from the screen
    Select_Option_Label.grid_forget()
    Input_Sales_Frame.grid_forget()
    Naive_Frame.grid_forget()
    WMA_Frame.grid_forget()
    EMA_Frame.grid_forget()
    
    #MA Frame
    MA_Frame.grid(row= 2, column= 0, columnspan= 5)
    
    #MA Month amount input
    MA_Month_Amount_Label = Label(MA_Frame, text= "Enter the number of Months to be Averaged: ")
    MA_Month_Amount_Label.grid(row = 3, column = 0)
    MA_Month_Amount_Input = Entry(MA_Frame)
    MA_Month_Amount_Input.grid(row = 3, column = 1)
    MA_Val_Submit = Button(MA_Frame, text="Submit", command= lambda: ValidateMA())
    MA_Val_Submit.grid(row = 4, column= 0, columnspan=5)
    
    #Validates Input
    def ValidateMA():
        try:
            MA_Error_Message = Label(MA_Frame, text="Edit this depending on situation", fg="red", bg="lightgray")
            MA_Val = MA_Month_Amount_Input.get()
            MA_Val = int(MA_Val)
            if MA_Val >= 12:
                MA_Error_Message.config(text="Please enter a value between 1-12")
                MA_Error_Message.grid(row=2, column = 0, columnspan= 5)
            #Calculation
            #copy to keep original safe
            MA_df = df.copy()
            
            MA_df["MA_Forecast"] = MA_df["Year_Sales_2021"].rolling(window=MA_Val).mean()
            MA_df["MA_Forecast"] = MA_df["MA_Forecast"].shift(1)
            MA_df["MA_Error"] = (MA_df["2022_Actual_Sales"] - MA_df["MA_Forecast"]).abs()
            MA_MAD = MA_df["MA_Error"].mean()
            
            #frame for TreeView
            Display_Data_Frame.grid(row= 0, column= 0, columnspan= 5)
            #TreeView Widget
            
            tv2.delete(*tv2.get_children())
            
            tv2["column"] = list(MA_df.columns)
            tv2["show"] = "headings"
            for column in tv2["column"]:
                tv2.heading(column, text = column)

            MA_df_rows = MA_df.to_numpy().tolist()
            for row in MA_df_rows:
                tv2.insert("", "end", values= row)
            tv2.grid(row= 100, column= 0)
            
            
            #Buttons After validation

            MA_MAD_Label.config(text= f"Moving Average MAD: {MA_MAD}")
            MA_MAD_Label.grid(row=5, column= 0)
            Graph_Comparison_Button.grid(row=5, column= 3)
            
            global MACompareGraph
            def MACompareGraph():
                #Moving Average Forecasting Graph
                x= np.array([0,1,2,3,4,5,6,7,8,9,10,11])

                plt.figure(figsize=(10,5), dpi=100)
                plt.plot(MA_df["2022_Actual_Sales"], marker=".", markersize="12", markeredgecolor="blue", label="Actual Sales 2022")
                plt.plot(MA_df["MA_Forecast"], marker=".", markersize="12", markeredgecolor="orange", label=" MA Forecast")
                plt.xticks(x,df.Months, rotation=50)

                plt.title(f"Actual Sales of 2022 vs {str(MA_Val)} Month MA Forecasted", fontdict={"fontsize": 15})
                plt.xlabel("Month")
                plt.ylabel("Sales")
                plt.legend()
                plt.legend(bbox_to_anchor=(0.85, 1.1), loc='upper left', borderaxespad=0)

                plt.show()
                
        except ValueError:
            MA_Error_Message.config(text="Please Enter an Integer Input")
            MA_Error_Message.grid(row=2, column = 0, columnspan= 5)
            
############################################################################
#Moving Average Section
def WMASection():
    root.geometry(Window_Size_After_Inputs)
    
    #removes anything that isnt for this tab from the screen
    Select_Option_Label.grid_forget()
    Input_Sales_Frame.grid_forget()
    Naive_Frame.grid_forget()
    MA_Frame.grid_forget()
    EMA_Frame.grid_forget()
    
    #WMA Frame
    WMA_Frame.grid(row= 2, column= 0, columnspan= 5)
    
    #MA Month amount input
    WMA_Month_Amount_Label = Label(WMA_Frame, text= "Enter the number of Months to be Averaged: ")
    WMA_Month_Amount_Label.grid(row = 3, column = 0)
    WMA_Month_Amount_Input = Entry(WMA_Frame)
    WMA_Month_Amount_Input.grid(row = 3, column = 1)
    WMA_Months_Submit = Button(WMA_Frame, text="Submit", command= lambda: ShowWeightsInputs())
    WMA_Months_Submit.grid(row = 10, column= 0, columnspan=5)
    
    #Displays weights inputs after inputting months averaged number
    def ShowWeightsInputs():
        Weights_Label = Label(WMA_Frame, text="Input Weights (Between 0 and 1):")
        Weights_Label.grid(row= 4, column= 0, columnspan=5)
        global WMA_Inputs_d
        WMA_Inputs_d = {}
        try:
            for i in range(int(WMA_Month_Amount_Input.get())):
                WMA_Inputs_d["WMA_Weights_Input{0}".format(i)] = Entry(WMA_Frame)
                WMA_Inputs_d["WMA_Weights_Input{0}".format(i)].grid(row= 5+ i, column= 0, columnspan=5)
                WMA_Months_Submit.config(command= lambda: SubmitWeights())
        except ValueError:
            
            
    def SubmitWeights():
        WMA_Input_Getter_d = {}
        Getter_List = []
        for i in range(int(WMA_Month_Amount_Input.get())):
            WMA_Input_Getter_d["Gotten_Input{0}".format(i)] = WMA_Inputs_d["WMA_Weights_Input{0}".format(i)].get()
            Getter_List.append(list(WMA_Input_Getter_d.values())[i])
            
        Getter_List = [int(x) for x in Getter_List]   
        if sum(Getter_List) != 1:
            print("thaas the wroongg numbeee")
        else:
            print("thas a good boi")
        
        print(sum(Getter_List))
        

############################################################################
#Moving Average Section
def ExponentialSection():
    #removes anything that isnt for this tab from the screen
    Select_Option_Label.grid_forget()
    Input_Sales_Frame.grid_forget()
    Naive_Frame.grid_forget()
    MA_Frame.grid_forget()
    WMA_Frame.grid_forget()
    
    #EMA Frame
    EMA_Frame.grid(row= 2, column= 0, columnspan= 5)
    pass
############################################################################
root.geometry("503x523")
root.mainloop()

In [44]:
df

,Months,Year_Sales_2021,2022_Actual_Sales
0,January,68,185
1,February,175,125
2,March,138,128
3,April,35,111
4,May,87,158
5,June,14,97
6,July,141,69
7,August,57,59
8,September,24,60
9,October,159,47


## Backend without GUI 

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [13]:
data = {
    'Months': ['January','February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'],
    'Year_Sales': ['-','-','-','-','-','-','-','-','-','-','-','-'],
    #for MAD error calculations
    '2022_Actual_Sales': ['-','-','-','-','-','-','-','-','-','-','-','-']
}

In [14]:
df = pd.DataFrame(data= data)
df

,Months,Year_Sales,2022_Actual_Sales
0,January,-,-
1,February,-,-
2,March,-,-
3,April,-,-
4,May,-,-
5,June,-,-
6,July,-,-
7,August,-,-
8,September,-,-
9,October,-,-


In [15]:
for i in range(12): 
    intcheck = False
    while intcheck == False:
        print("Enter", df["Months"][i], end=" ")
        try:
            userInput = int(input("Sales: "))
        except ValueError:
            print('Thats the wrong numba')
        else:
            intcheck = True
            df["Year_Sales"][i] = userInput

Enter January Sales: 200
Enter February Sales: 200
Enter March Sales: 200
Enter April Sales: 200
Enter May Sales: 200
Enter June Sales: 200
Enter July Sales: 200
Enter August Sales: 200
Enter September Sales: 200
Enter October Sales: 200
Enter November Sales: 200
Enter December Sales: 200


In [16]:
df

,Months,Year_Sales,2022_Actual_Sales
0,January,200,-
1,February,200,-
2,March,200,-
3,April,200,-
4,May,200,-
5,June,200,-
6,July,200,-
7,August,200,-
8,September,200,-
9,October,200,-


In [17]:
x= np.array([0,1,2,3,4,5,6,7,8,9,10,11])

plt.figure(figsize=(5,3), dpi=100)
plt.plot(df["Year_Sales"], marker=".", markersize="12", markeredgecolor="red", label="Actual Sales")
plt.xticks(x,df["Months"], rotation=50)

plt.title("Actual Sales of 2021", fontdict={"fontsize": 15})
plt.xlabel("Month")
plt.ylabel("Sales")

plt.show()

## Naive

In [23]:
df["Naive_Forcast"] = df["Year_Sales"]
df

,Months,Year_Sales,2022_Actual_Sales,Naive_Forcast
0,January,200,-,200
1,February,200,-,200
2,March,200,-,200
3,April,200,-,200
4,May,200,-,200
5,June,200,-,200
6,July,200,-,200
7,August,200,-,200
8,September,200,-,200
9,October,200,-,200


## Moving Average

In [206]:
def MA_value(value):
  while True:
    try:
       MA_val = int(input(value))       
    except ValueError:
       print("Not an integer! Try again.")
       continue
    if MA_val >= 13:
        print("Enter a smaller Number (Between 1 - 11)")
        continue
    else:
       return MA_val 
       break 
     
MA_val = MA_value("Enter a number of Months \n for Moving Average (1 - 11): ")

Enter a number of Months 
 for Moving Average (1 - 11): 12


In [207]:
df["MA_Forecast"] = df["Year_Sales"].rolling(window=MA_val).mean()
df

KeyError: 'Year_Sales'

In [47]:
#shifts cells down
df["MA_Forecast"] = df["MA_Forecast"].shift(1)
df

,Months,Year_Sales,2022_Actual_Sales,Naive_Forcast,MA_Forecast
0,January,536,-,536,NaN
1,February,626,-,626,NaN
2,March,157,-,157,NaN
3,April,659,-,659,439.666667
4,May,294,-,294,480.666667
5,June,468,-,468,370.000000
6,July,478,-,478,473.666667
7,August,512,-,512,413.333333
8,September,690,-,690,486.000000
9,October,149,-,149,560.000000


In [48]:
#Moving Average Forecasting Graph
x= np.array([0,1,2,3,4,5,6,7,8,9,10,11])

plt.figure(figsize=(5,3), dpi=100)
plt.plot(df["Year_Sales"], marker=".", markersize="12", markeredgecolor="blue", label="Actual Sales 2021")
plt.plot(df["MA_Forecast"], marker=".", markersize="12", markeredgecolor="orange", label=" MA Forecast")
plt.xticks(x,df.Months, rotation=50)

plt.title(f"Actual Sales of 2021 vs {str(MA_val)} Month MA Forecasted", fontdict={"fontsize": 15})
plt.xlabel("Month")
plt.ylabel("Sales")
plt.legend()
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)

# plt.plot(df.Sales_2022)

plt.show()

## Weighted Moving Average

In [49]:
def WMA_value(value):
  while True:
    try:
       WMA_val = int(input(value))       
    except ValueError:
       print("Not an integer! Try again.")
       continue
    if MA_val >= 12:
        print("Enter a smaller Number (Between 1 - 11)")
        continue
    else:
       return MA_val 
       break 
     
WMA_val = MA_value("Enter a number of Months \n for WMA and Weight of each Month before (1 - 11): ")


Enter a number of Months 
 for WMA and Weight of each Month before (1 - 11): 3


In [50]:
weights = []
for i in range(WMA_val):
    print(df["Months"][i], end=" ")
    print("Weight Value ", end=" ")
    weight = float(input(": "))
    weights.append(weight)

January Weight Value  : 0.5
February Weight Value  : 0.4
March Weight Value  : 0.1


In [51]:
weights

[0.5, 0.4, 0.1]

In [52]:
df["WMA_Forecast"] = df["Year_Sales"].rolling(window = WMA_val).apply(lambda x: np.sum(weights*x)/np.sum(weights))
#shifts cells down
df["WMA_Forecast"] = df["WMA_Forecast"].shift(1)
df

,Months,Year_Sales,2022_Actual_Sales,Naive_Forcast,MA_Forecast,WMA_Forecast
0,January,536,-,536,NaN,NaN
1,February,626,-,626,NaN,NaN
2,March,157,-,157,NaN,NaN
3,April,659,-,659,439.666667,534.1
4,May,294,-,294,480.666667,441.7
5,June,468,-,468,370.000000,371.5
6,July,478,-,478,473.666667,493.9
7,August,512,-,512,413.333333,382.0
8,September,690,-,690,486.000000,476.4
9,October,149,-,149,560.000000,512.8


In [53]:
df

,Months,Year_Sales,2022_Actual_Sales,Naive_Forcast,MA_Forecast,WMA_Forecast
0,January,536,-,536,NaN,NaN
1,February,626,-,626,NaN,NaN
2,March,157,-,157,NaN,NaN
3,April,659,-,659,439.666667,534.1
4,May,294,-,294,480.666667,441.7
5,June,468,-,468,370.000000,371.5
6,July,478,-,478,473.666667,493.9
7,August,512,-,512,413.333333,382.0
8,September,690,-,690,486.000000,476.4
9,October,149,-,149,560.000000,512.8


In [54]:
def WMA_Graph():

    #Weighted Moving Average Forecasting Graph
    x= np.array([0,1,2,3,4,5,6,7,8,9,10,11])

    plt.figure(figsize=(5,3), dpi=100)
    plt.plot(df["Year_Sales"], marker=".", markersize="12", markeredgecolor="blue", label="Actual Sales 2021")
    plt.plot(df["WMA_Forecast"], marker=".", markersize="12", markeredgecolor="orange", label="WMA Forecast")
    plt.xticks(x,df.Months, rotation=50)

    plt.title(f"Actual Sales of 2021 vs {str(WMA_val)} Month WMA Forecasted", fontdict={"fontsize": 15})
    plt.xlabel("Month")
    plt.ylabel("Sales")
    plt.legend()
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)


    plt.show()

WMA_Graph()

## Exponenial Forecasting

In [55]:
#0.429483326 
def EMU_Calc(alpha_input):

    df["ema_Forecast"] = df["Year_Sales"].ewm(alpha = alpha_input, adjust=False).mean()
    print(df["ema_Forecast"])

#call the function
EMU_Calc(float(input("Choose Optimal Alpha: ")))

Choose Optimal Alpha: 0.1424
0     536.000000
1     548.816000
2     493.021402
3     516.656754
4     484.950432
5     482.536691
6     481.890666
7     486.178235
8     515.202454
9     463.055225
10    412.922561
11    459.498388
Name: ema_Forecast, dtype: float64


In [56]:
#EMA Forecasting Graph
def EMU_Graph():
    x= np.array([0,1,2,3,4,5,6,7,8,9,10,11])

    plt.figure(figsize=(5,3), dpi=100)
    plt.plot(df["Year_Sales"], marker=".", markersize="12", markeredgecolor="blue", label="Actual Sales 2022")
    plt.plot(df["ema_Forecast"], marker=".", markersize="12", markeredgecolor="orange", label=" EMA Forecast")
    plt.xticks(x,df["Months"], rotation=50)

    plt.title("Actual Sales of 2022 vs EMA Forecasted", fontdict={"fontsize": 15})
    plt.xlabel("Month")
    plt.ylabel("Sales")

    plt.legend()
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
    # plt.plot(df.Sales_2022)

    plt.show()

#call the function
EMU_Graph()

#### Errors and MAD

In [57]:
df = df[["Months", "Year_Sales", "Naive_Forcast", "MA_Forecast", "WMA_Forecast", "ema_Forecast","2022_Actual_Sales"]]
df

,Months,Year_Sales,Naive_Forcast,MA_Forecast,WMA_Forecast,ema_Forecast,2022_Actual_Sales
0,January,536,536,NaN,NaN,536.000000,-
1,February,626,626,NaN,NaN,548.816000,-
2,March,157,157,NaN,NaN,493.021402,-
3,April,659,659,439.666667,534.1,516.656754,-
4,May,294,294,480.666667,441.7,484.950432,-
5,June,468,468,370.000000,371.5,482.536691,-
6,July,478,478,473.666667,493.9,481.890666,-
7,August,512,512,413.333333,382.0,486.178235,-
8,September,690,690,486.000000,476.4,515.202454,-
9,October,149,149,560.000000,512.8,463.055225,-


In [58]:
#for actual 2022 Sales
for i in range(12): 
    intcheck = False
    while intcheck == False:
        print("Enter", df["Months"][i], end=" ")
        try:
            userInput = int(input("Real Sales: "))
        except ValueError:
            print('Thats the wrong numba')
        else:
            intcheck = True
            df["2022_Actual_Sales"][i] = userInput

Enter January Real Sales: 600


C:\Users\Ahmed\AppData\Local\Temp/ipykernel_10244/2485808213.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["2022_Actual_Sales"][i] = userInput


Enter February Real Sales: 
Thats the wrong numba
Enter February Real Sales: 
Thats the wrong numba
Enter February Real Sales: 
Thats the wrong numba
Enter February Real Sales: dsg
Thats the wrong numba
Enter February Real Sales: 700
Enter March Real Sales: 200
Enter April Real Sales: 800
Enter May Real Sales: 300
Enter June Real Sales: 500
Enter July Real Sales: 500
Enter August Real Sales: 600
Enter September Real Sales: 700
Enter October Real Sales: 300
Enter November Real Sales: 200
Enter December Real Sales: 800


In [59]:
df

,Months,Year_Sales,Naive_Forcast,MA_Forecast,WMA_Forecast,ema_Forecast,2022_Actual_Sales
0,January,536,536,NaN,NaN,536.000000,600
1,February,626,626,NaN,NaN,548.816000,700
2,March,157,157,NaN,NaN,493.021402,200
3,April,659,659,439.666667,534.1,516.656754,800
4,May,294,294,480.666667,441.7,484.950432,300
5,June,468,468,370.000000,371.5,482.536691,500
6,July,478,478,473.666667,493.9,481.890666,500
7,August,512,512,413.333333,382.0,486.178235,600
8,September,690,690,486.000000,476.4,515.202454,700
9,October,149,149,560.000000,512.8,463.055225,300


In [60]:
error_df = df.copy()
error_df

,Months,Year_Sales,Naive_Forcast,MA_Forecast,WMA_Forecast,ema_Forecast,2022_Actual_Sales
0,January,536,536,NaN,NaN,536.000000,600
1,February,626,626,NaN,NaN,548.816000,700
2,March,157,157,NaN,NaN,493.021402,200
3,April,659,659,439.666667,534.1,516.656754,800
4,May,294,294,480.666667,441.7,484.950432,300
5,June,468,468,370.000000,371.5,482.536691,500
6,July,478,478,473.666667,493.9,481.890666,500
7,August,512,512,413.333333,382.0,486.178235,600
8,September,690,690,486.000000,476.4,515.202454,700
9,October,149,149,560.000000,512.8,463.055225,300


In [61]:
#Naive Error
error_df["Naive_Error"] = (df["2022_Actual_Sales"] - df["Naive_Forcast"]).abs()
error_df

,Months,Year_Sales,Naive_Forcast,MA_Forecast,WMA_Forecast,ema_Forecast,2022_Actual_Sales,Naive_Error
0,January,536,536,NaN,NaN,536.000000,600,64
1,February,626,626,NaN,NaN,548.816000,700,74
2,March,157,157,NaN,NaN,493.021402,200,43
3,April,659,659,439.666667,534.1,516.656754,800,141
4,May,294,294,480.666667,441.7,484.950432,300,6
5,June,468,468,370.000000,371.5,482.536691,500,32
6,July,478,478,473.666667,493.9,481.890666,500,22
7,August,512,512,413.333333,382.0,486.178235,600,88
8,September,690,690,486.000000,476.4,515.202454,700,10
9,October,149,149,560.000000,512.8,463.055225,300,151


In [63]:
Naive_MAD = error_df["Naive_Error"].mean()
print(f"Naive MAD: {Naive_MAD}")

Naive MAD: 65.0


In [66]:
#Naive Compare
x= np.array([0,1,2,3,4,5,6,7,8,9,10,11])

plt.figure(figsize=(5,3), dpi=100)
plt.plot(error_df["2022_Actual_Sales"], marker=".", markersize="12", markeredgecolor="blue", label="Actual Sales 2022")
plt.plot(error_df["Naive_Forcast"], marker=".", markersize="12", markeredgecolor="orange", label="Naive Forecasting")
plt.xticks(x, error_df["Months"], rotation=50)

plt.title("Actual Sales of 2022 vs Naive Forecasted", fontdict={"fontsize": 15})
plt.xlabel("Month")
plt.ylabel("Sales")
plt.legend()
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)


plt.show()

In [67]:
#MA Error
error_df["MA_Error"] = (df["2022_Actual_Sales"] - df["MA_Forecast"]).abs()
error_df

,Months,Year_Sales,Naive_Forcast,MA_Forecast,WMA_Forecast,ema_Forecast,2022_Actual_Sales,Naive_Error,MA_Error
0,January,536,536,NaN,NaN,536.000000,600,64,NaN
1,February,626,626,NaN,NaN,548.816000,700,74,NaN
2,March,157,157,NaN,NaN,493.021402,200,43,NaN
3,April,659,659,439.666667,534.1,516.656754,800,141,360.333333
4,May,294,294,480.666667,441.7,484.950432,300,6,180.666667
5,June,468,468,370.000000,371.5,482.536691,500,32,130.0
6,July,478,478,473.666667,493.9,481.890666,500,22,26.333333
7,August,512,512,413.333333,382.0,486.178235,600,88,186.666667
8,September,690,690,486.000000,476.4,515.202454,700,10,214.0
9,October,149,149,560.000000,512.8,463.055225,300,151,260.0


In [70]:
MA_MAD = error_df["MA_Error"].mean()
print(f"Moving Average MAD: {round(MA_MAD,2)}")

Moving Average MAD: 232.41


In [78]:
#MA Error
x= np.array([0,1,2,3,4,5,6,7,8,9,10,11])

plt.figure(figsize=(5,3), dpi=100)
plt.plot(error_df["2022_Actual_Sales"], marker=".", markersize="12", markeredgecolor="blue", label="Actual Sales 2022")
plt.plot(error_df["MA_Error"], marker=".", markersize="12", markeredgecolor="orange", label=" MA Forecast")
plt.xticks(x,error_df["Months"], rotation=50)

plt.title("Actual Sales of 2022 vs 5 Month MA Forecasted", fontdict={"fontsize": 15})
plt.xlabel("Month")
plt.ylabel("Sales")
plt.legend()
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)


plt.show()

In [75]:
#WMA Error
error_df["WMA_Error"] = (error_df["2022_Actual_Sales"] - error_df["WMA_Forecast"]).abs()
error_df

,Months,Year_Sales,Naive_Forcast,MA_Forecast,WMA_Forecast,ema_Forecast,2022_Actual_Sales,Naive_Error,MA_Error,WMA_Error
0,January,536,536,NaN,NaN,536.000000,600,64,NaN,NaN
1,February,626,626,NaN,NaN,548.816000,700,74,NaN,NaN
2,March,157,157,NaN,NaN,493.021402,200,43,NaN,NaN
3,April,659,659,439.666667,534.1,516.656754,800,141,360.333333,265.9
4,May,294,294,480.666667,441.7,484.950432,300,6,180.666667,141.7
5,June,468,468,370.000000,371.5,482.536691,500,32,130.0,128.5
6,July,478,478,473.666667,493.9,481.890666,500,22,26.333333,6.1
7,August,512,512,413.333333,382.0,486.178235,600,88,186.666667,218.0
8,September,690,690,486.000000,476.4,515.202454,700,10,214.0,223.6
9,October,149,149,560.000000,512.8,463.055225,300,151,260.0,212.8


In [76]:
WMA_MAD = error_df["WMA_Error"].mean()
print(f"Weighted Moving Average MAD: {round(WMA_MAD,2)}")

Weighted Moving Average MAD: 214.2


In [79]:
#WMA Error
x= np.array([0,1,2,3,4,5,6,7,8,9,10,11])

plt.figure(figsize=(5,3), dpi=100)
plt.plot(error_df["2022_Actual_Sales"], marker=".", markersize="12", markeredgecolor="blue", label="Actual Sales 2022")
plt.plot(error_df["WMA_Error"], marker=".", markersize="12", markeredgecolor="orange", label="WMA Forecast")
plt.xticks(x,error_df["Months"], rotation=50)

plt.title("Actual Sales of 2022 vs 5 Month MA Forecasted", fontdict={"fontsize": 15})
plt.xlabel("Month")
plt.ylabel("Sales")
plt.legend()
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)


plt.show()

In [84]:
#Exponential Error
error_df["EMA_Error"] = (error_df["2022_Actual_Sales"] - error_df["ema_Forecast"]).abs()
error_df

,Months,Year_Sales,Naive_Forcast,MA_Forecast,WMA_Forecast,ema_Forecast,2022_Actual_Sales,Naive_Error,MA_Error,WMA_Error,ema_Error
0,January,536,536,NaN,NaN,536.000000,600,64,NaN,NaN,64.0
1,February,626,626,NaN,NaN,548.816000,700,74,NaN,NaN,151.184
2,March,157,157,NaN,NaN,493.021402,200,43,NaN,NaN,293.021402
3,April,659,659,439.666667,534.1,516.656754,800,141,360.333333,265.9,283.343246
4,May,294,294,480.666667,441.7,484.950432,300,6,180.666667,141.7,184.950432
5,June,468,468,370.000000,371.5,482.536691,500,32,130.0,128.5,17.463309
6,July,478,478,473.666667,493.9,481.890666,500,22,26.333333,6.1,18.109334
7,August,512,512,413.333333,382.0,486.178235,600,88,186.666667,218.0,113.821765
8,September,690,690,486.000000,476.4,515.202454,700,10,214.0,223.6,184.797546
9,October,149,149,560.000000,512.8,463.055225,300,151,260.0,212.8,163.055225


In [86]:
EMA_MAD = error_df["EMA_Error"].mean()
print(f"Exponential Smoothing MAD: {round(EMA_MAD,2)}")

Exponential Smoothing MAD: 168.93


In [87]:
#EMA Error
x= np.array([0,1,2,3,4,5,6,7,8,9,10,11])

plt.figure(figsize=(5,3), dpi=100)
plt.plot(error_df["2022_Actual_Sales"], marker=".", markersize="12", markeredgecolor="blue", label="Actual Sales 2022")
plt.plot(error_df["EMA_Error"], marker=".", markersize="12", markeredgecolor="orange", label="WMA Forecast")
plt.xticks(x,error_df["Months"], rotation=50)

plt.title("Actual Sales of 2022 vs 5 Month MA Forecasted", fontdict={"fontsize": 15})
plt.xlabel("Month")
plt.ylabel("Sales")
plt.legend()
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)


plt.show()

In [66]:
#Save CSV
df.to_csv('ToCSV.csv', index=False)